<a href="https://colab.research.google.com/github/tesla1618/Algebros/blob/main/3Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import os
import pandas as pd
import numpy as np
import json
import math
import librosa

In [2]:
model_name = 'cnn_7738_BeatsAI.keras'
path = F"/content/drive/MyDrive/SavedModels/{model_name}"

In [3]:
dataDir = "/content/drive/MyDrive/Data/genres_original"
jsonPath = "/content/drive/MyDrive/Data/data.json"
csvPath = "/content/drive/MyDrive/Data/features_30_sec.csv"

In [4]:
with open(jsonPath, "r") as fp:
  data = json.load(fp)

X = np.array(data["mfcc"])
y = np.array(data["labels"])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
X_train.shape

(6605, 130, 13)

In [7]:
X_train.shape[2]

13

In [8]:
modelLSTM = keras.Sequential()


modelLSTM.add(keras.layers.LSTM(64,input_shape = (X_train.shape[1], X_train.shape[2]), return_sequences=True))
modelLSTM.add(keras.layers.LSTM(128, return_sequences=True,kernel_regularizer=keras.regularizers.l2(0.01)))
modelLSTM.add(keras.layers.LSTM(64, return_sequences=True,kernel_regularizer=keras.regularizers.l2(0.01)))
modelLSTM.add(keras.layers.LSTM(64))


modelLSTM.add(keras.layers.Dense(128, activation='relu'))
modelLSTM.add(keras.layers.Dropout(0.3))

modelLSTM.add(keras.layers.Dense(64, activation='relu'))
modelLSTM.add(keras.layers.Dropout(0.3))

modelLSTM.add(keras.layers.Dense(32, activation='relu'))
modelLSTM.add(keras.layers.Dropout(0.3))

modelLSTM.add(keras.layers.Dense(10, activation='softmax'))

In [9]:
modelCNN = keras.Sequential()

modelCNN.add(keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
modelCNN.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
modelCNN.add(keras.layers.BatchNormalization())

modelCNN.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
modelCNN.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
modelCNN.add(keras.layers.BatchNormalization())


modelCNN.add(keras.layers.Conv2D(64, (2, 2), activation='relu'))
modelCNN.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
modelCNN.add(keras.layers.BatchNormalization())

modelCNN.add(keras.layers.Flatten())

modelCNN.add(keras.layers.Dense(64, activation='relu'))
modelCNN.add(keras.layers.Dropout(0.2))

modelCNN.add(keras.layers.Dense(10, activation='softmax'))

In [63]:
modelNN = keras.Sequential([
    keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),
    keras.layers.Dense(2048, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1024, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

In [69]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

In [70]:
modelLSTM.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
modelCNN.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [65]:
modelNN.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [66]:
modelLSTM.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 130, 64)           19968     
                                                                 
 lstm_1 (LSTM)               (None, 130, 128)          98816     
                                                                 
 lstm_2 (LSTM)               (None, 130, 64)           49408     
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8

In [16]:
modelCNN.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 11, 128)      1280      
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 6, 128)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 64, 6, 128)        512       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 4, 128)        147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 31, 2, 128)        0         
 g2D)                                                            
                                                      

In [17]:
modelNN.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 1690)              0         
                                                                 
 dense_6 (Dense)             (None, 2048)              3463168   
                                                                 
 dropout_4 (Dropout)         (None, 2048)              0         
                                                                 
 dense_7 (Dense)             (None, 1024)              2098176   
                                                                 
 dropout_5 (Dropout)         (None, 1024)              0         
                                                                 
 dense_8 (Dense)             (None, 512)               524800    
                                                                 
 dropout_6 (Dropout)         (None, 512)              

In [72]:
historyNN = modelNN.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=100)

Epoch 1/100
207/207 [==============================] - 1s 7ms/step - loss: 5.7330 - accuracy: 0.3625 - val_loss: 5.5595 - val_accuracy: 0.4405
Epoch 2/100
207/207 [==============================] - 1s 7ms/step - loss: 5.6134 - accuracy: 0.3803 - val_loss: 5.4361 - val_accuracy: 0.4468
Epoch 3/100
207/207 [==============================] - 1s 7ms/step - loss: 5.4911 - accuracy: 0.4012 - val_loss: 5.3253 - val_accuracy: 0.4744
Epoch 4/100
207/207 [==============================] - 1s 7ms/step - loss: 5.3982 - accuracy: 0.4073 - val_loss: 5.2442 - val_accuracy: 0.4800
Epoch 5/100
207/207 [==============================] - 2s 8ms/step - loss: 5.2943 - accuracy: 0.4179 - val_loss: 5.1288 - val_accuracy: 0.4984
Epoch 6/100
207/207 [==============================] - 2s 8ms/step - loss: 5.1975 - accuracy: 0.4256 - val_loss: 5.0688 - val_accuracy: 0.5115
Epoch 7/100
207/207 [==============================] - 1s 6ms/step - loss: 5.1040 - accuracy: 0.4380 - val_loss: 4.9635 - val_accuracy: 0.5012

In [19]:
historyCNN = modelCNN.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=100)

Epoch 1/100
207/207 [==============================] - 17s 13ms/step - loss: 1.8591 - accuracy: 0.3764 - val_loss: 1.3880 - val_accuracy: 0.5210
Epoch 2/100
207/207 [==============================] - 2s 8ms/step - loss: 1.3376 - accuracy: 0.5375 - val_loss: 1.1642 - val_accuracy: 0.5853
Epoch 3/100
207/207 [==============================] - 2s 7ms/step - loss: 1.1739 - accuracy: 0.5926 - val_loss: 1.0468 - val_accuracy: 0.6341
Epoch 4/100
207/207 [==============================] - 2s 8ms/step - loss: 1.0209 - accuracy: 0.6379 - val_loss: 1.0434 - val_accuracy: 0.6379
Epoch 5/100
207/207 [==============================] - 2s 8ms/step - loss: 0.9234 - accuracy: 0.6798 - val_loss: 0.9303 - val_accuracy: 0.6711
Epoch 6/100
207/207 [==============================] - 2s 8ms/step - loss: 0.8393 - accuracy: 0.7084 - val_loss: 0.9746 - val_accuracy: 0.6422
Epoch 7/100
207/207 [==============================] - 2s 10ms/step - loss: 0.7647 - accuracy: 0.7364 - val_loss: 0.8457 - val_accuracy: 0.7

In [71]:
historyLSTM = modelLSTM.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=100)

Epoch 1/100


Exception ignored in: <function ConcreteFunctionGarbageCollector.__del__ at 0x78a5d53492d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/monomorphic_function.py", line 1892, in __del__
    def __del__(self):
KeyboardInterrupt: 


207/207 [==============================] - 294s 31ms/step - loss: 4.4738 - accuracy: 0.1936 - val_loss: 3.8077 - val_accuracy: 0.3370
Epoch 2/100
207/207 [==============================] - 5s 26ms/step - loss: 3.4356 - accuracy: 0.2980 - val_loss: 2.9179 - val_accuracy: 0.4041
Epoch 3/100
207/207 [==============================] - 6s 29ms/step - loss: 2.8005 - accuracy: 0.3520 - val_loss: 2.4641 - val_accuracy: 0.4264
Epoch 4/100
207/207 [==============================] - 5s 23ms/step - loss: 2.4282 - accuracy: 0.3759 - val_loss: 2.2360 - val_accuracy: 0.4299
Epoch 5/100
207/207 [==============================] - 6s 31ms/step - loss: 2.2060 - accuracy: 0.3977 - val_loss: 2.0145 - val_accuracy: 0.4479
Epoch 6/100
207/207 [==============================] - 5s 23ms/step - loss: 2.0511 - accuracy: 0.4164 - val_loss: 1.8908 - val_accuracy: 0.4730
Epoch 7/100
207/207 [==============================] - 5s 24ms/step - loss: 1.9583 - accuracy: 0.4253 - val_loss: 1.7881 - val_accuracy: 0.4938
Ep